In [ ]:
%run -i ../startup.py
ENTITY_TYPE = 'work'

Wikidata entities:

https://www.wikidata.org/wiki/Q2188189 musical work

Wikidata properties:

https://www.wikidata.org/wiki/Property:P86 composer

https://www.wikidata.org/wiki/Property:P870 instrumentation

https://www.wikidata.org/wiki/Property:P214 VIAF

https://www.wikidata.org/wiki/Property:P268 BNF

https://www.wikidata.org/wiki/Property:P244 LoC

https://www.wikidata.org/wiki/Property:P839 IMSLP

https://www.wikidata.org/wiki/Property:P435 MusicBrainz work ID

https://www.wikidata.org/wiki/Property:P1994 AllMusic composition ID

https://www.wikidata.org/wiki/Property:P5229 Carnegie Hall work ID

https://www.wikidata.org/wiki/Property:P6080 Discogs composition ID (obsoleted by discogs in October 2019)

examples

https://www.wikidata.org/wiki/Q3478907

## Works from Wikidata

In [ ]:
links_type_from_wd = sparql("""
SELECT distinct (count(?work) as ?cnt) ?ins ?insLabel
WHERE {
  ?work wdt:P31 ?ins;
    wdt:P435 ?mbid.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
group by ?ins ?insLabel
order by DESC(?cnt)
""")
links_type_from_wd[links_type_from_wd.cnt.astype(int) > 10]

In [ ]:
sparql("""
SELECT (COUNT(?work) AS ?cnt)
WHERE {
  ?work wdt:P435 ?mbid.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
""")

In [ ]:
# linked to MB work
links_from_wd = sparql("""
SELECT (?work AS ?wd) ?mbid ?workLabel (GROUP_CONCAT(?categoryLabel; SEPARATOR=", ") AS ?workType)
WHERE {
  ?work wdt:P435 ?mbid .
  ?work wdt:P31 ?category .
  ?category rdfs:label ?categoryLabel FILTER(LANG(?categoryLabel) = "en")
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
GROUP BY ?work ?mbid ?workLabel
ORDER BY ASC(?workType) ASC(?workLabel)
""")
links_from_wd.rename(columns={'workLabel': 'name'}, inplace=True)
print('Count:', len(links_from_wd))
display_df(links_from_wd.head())

## Works from MusicBrainz with wikidata links

In [ ]:
links_from_mb = sql("""
SELECT
    url.url AS wd,
    work.gid AS mbid,
    work.name
FROM work
JOIN l_url_work AS luw ON luw.entity1 = work.id
JOIN url               ON luw.entity0 = url.id
WHERE
    url.url LIKE '%%wikidata.org%%'
ORDER BY work.name
;
""")
links_from_mb.wd = links_from_mb.wd.apply(lambda s: s.split('/')[-1])
links_from_mb.mbid = links_from_mb.mbid.apply(str)

print('Count:', len(links_from_mb))
display_df(links_from_mb.head())

## Duplicate links

In [ ]:
duplicate_wd = links_from_mb[[
    'wd', 'mbid', 'name']].groupby('wd').filter(
    lambda row: len(row.mbid) > 1).sort_values('wd')

print('Count:', len(duplicate_wd))
display_df(duplicate_wd.head())

In [ ]:
duplicate_mb = links_from_mb[['wd', 'mbid', 'name']].groupby('mbid').filter(
    lambda row: len(row.mbid) > 1).sort_values('mbid')

print('Count:', len(duplicate_mb))
display_df(duplicate_mb.head())

## Data alignment

In [ ]:
merge = pd.merge(links_from_wd, links_from_mb, 
                 on=['wd', 'mbid'], suffixes=('_wd', '_mb'),
                 how='outer', indicator=True)
display_df(merge.head())

In [ ]:
# link in mb but missing in wd
links_to_add_to_wd = merge.loc[lambda x : x['_merge']=='right_only'][[
    'name_mb', 'mbid', 'wd']]

print('Count:', len(links_to_add_to_wd))
display_df(links_to_add_to_wd.head())

In [ ]:
# link in wd but missing in mb
links_to_add_to_mb = merge.loc[lambda x : x['_merge']=='left_only'][[
    'name_wd', 'wd', 'mbid', 'workType']]
links_to_add_to_mb['edit_link'] = links_to_add_to_mb.apply(
    mb_work_edit_wd_link, axis=1)

print('Count:', len(links_to_add_to_mb))
display_df(links_to_add_to_mb.head())

## Report

In [ ]:
import jinja2

template = jinja2.Template("""
<!doctype html>

<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Alignment of MusicBrainz and Wikidata Works</title>
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  </head>

  <body style="margin: 20px;">
    <h1>Alignment of MusicBrainz and Wikidata Works</h1>

    <p>Latest MB database update: {{ MB_DATABASE_VERSION }}</p>
    <p>Latest update: {{ date.today().isoformat() }}</p>

    <ol>
      <li>
        <a href="#wddup">MusicBrainz entities sharing a Wikidata link</a>
        ({{ duplicate_wd.shape[0] }} rows)
      </li>
      <li>
        <a href="#mbdup">Wikidata entities sharing a MusicBrainz link</a>
        ({{ duplicate_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#wd2mb">Add missing Wikidata links to MusicBrainz</a>
        ({{ links_to_add_to_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#mb2wd">Add missing MusicBrainz links to Wikidata</a>
        ({{ links_to_add_to_wd.shape[0] }} rows)
      </li>
    </ol>
    
    <h2 id="wddup">MusicBrainz entities sharing a Wikidata link</h2>
    {{ df_to_html(duplicate_wd) }}

    <h2 id="mbdup">Wikidata entities sharing a MusicBrainz link</h2>
    {{ df_to_html(duplicate_mb) }}

    <h2 id="wd2mb">Add missing Wikidata links to MusicBrainz</h2>
    {{ df_to_html(links_to_add_to_mb) }}

    <h2 id="mb2wd">Add missing MusicBrainz links to Wikidata</h2>
    {{ df_to_html(links_to_add_to_wd) }}
  </body>
</html>
""")

with open('../docs/wd-works-report.html', 'w') as f:
    f.write(template.render(**globals())
            .replace('&lt;', '<').replace('&gt;', '>')
            .replace('class="dataframe"', 'class="table table-striped table-hover table-sm"')
            .replace('thead', 'thead class="thead-light"'))